In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import StratifiedShuffleSplit
import xgboost as xgb
import datetime
from pandas.api.types import CategoricalDtype

In [34]:
train = pd.read_csv('/Users/young/dataset_local/walmart-recruiting-trip-type-classification/train.csv',dtype=({'Upc':object,'FinelineNumber':object,'ScanCount':np.int16,'TripType':np.int16,'VisitNumber':np.int32}))
test = pd.read_csv('/Users/young/dataset_local/walmart-recruiting-trip-type-classification/test.csv',dtype=({'Upc':object,'FinelineNumber':object,'ScanCount':np.int16,'TripType':np.int16,'VisitNumber':np.int32}))

In [35]:
test['TripType'] = 0
test['TripType']

0         0
1         0
2         0
3         0
4         0
         ..
653641    0
653642    0
653643    0
653644    0
653645    0
Name: TripType, Length: 653646, dtype: int64

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647054 entries, 0 to 647053
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   TripType               647054 non-null  int16 
 1   VisitNumber            647054 non-null  int32 
 2   Weekday                647054 non-null  object
 3   Upc                    642925 non-null  object
 4   ScanCount              647054 non-null  int16 
 5   DepartmentDescription  645693 non-null  object
 6   FinelineNumber         642925 non-null  object
dtypes: int16(2), int32(1), object(4)
memory usage: 24.7+ MB


In [37]:
tr_VisitNumber = list(train.VisitNumber.unique())
te_VisitNumber = list(test.VisitNumber.unique())

In [38]:
train.Upc.fillna('0',inplace=True)
train.DepartmentDescription.fillna('NA',inplace=True)
train.FinelineNumber.fillna('-1',inplace=True)
test.Upc.fillna('0',inplace=True)
test.DepartmentDescription.fillna('NA',inplace=True)
test.FinelineNumber.fillna('-1',inplace=True)


In [39]:
train.Upc = train.Upc.astype(np.int64)
train.FinelineNumber = train.FinelineNumber.astype(np.int16)
train.ScanCount = train.ScanCount.astype(np.int32)
test.Upc = test.Upc.astype(np.int64)
test.FinelineNumber = test.FinelineNumber.astype(np.int16)
test.ScanCount = test.ScanCount.astype(np.int32)

In [49]:
train.sort_values(by='VisitNumber',inplace=True)
train[train['DepartmentDescription']=='PHARMACY RX']

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
1155,44,496,Friday,0,1,PHARMACY RX,-1
1216,5,521,Friday,0,1,PHARMACY RX,-1
1373,5,585,Friday,0,1,PHARMACY RX,-1
1457,5,619,Friday,0,1,PHARMACY RX,-1
1455,5,619,Friday,0,1,PHARMACY RX,-1
...,...,...,...,...,...,...,...
636715,5,188839,Sunday,0,1,PHARMACY RX,-1
636716,5,188839,Sunday,0,1,PHARMACY RX,-1
636717,5,188839,Sunday,0,1,PHARMACY RX,-1
636847,5,188896,Sunday,0,1,PHARMACY RX,-1


In [65]:
train_visitnumber_triptype = train.groupby(['VisitNumber']).agg({'TripType': 'first'}).reset_index()
train_visitnumber_triptype

,VisitNumber,TripType
0,5,999
1,7,30
2,8,26
3,9,8
4,10,8
...,...,...
95669,191343,25
95670,191344,22
95671,191345,39
95672,191346,39
